### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Aggregate Data --> Not Done Yet

4. Add Binary Class --> Done, Should Add Binary Class Later

5. Summary Statistics Features --> Done

6. Wrapper Functions --> Done, Need to Test Though

7. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

8. Sklearn Pipeline Numerical Features --> StandardScaler Done

9. Create Lagged Features --> Done

10. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

11. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

12. PySpark: XGBoost Classification Feature Importance

In [1]:
from Input_Variables.read_vars import raw_data_storage, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      ml_models_train_split, ml_models_test_split, model_storage_location, \
                                      time_series_lag_values_created

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Model_Creation.xgboost_model import XGBoost_Classification
from Model_Evaluation.classification_evaluation import Classification_Evalaution_Metrics
from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# XGBoost Model Module
xgboost_classification=XGBoost_Classification()

# Classification Evaluation
classification_evalaution_metrics=Classification_Evalaution_Metrics()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# PySpark

### 1. PySpark: Reading In Data

In [3]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/17 16:40:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 2. PySpark: Custom Imputation Pipeline

In [4]:
####### PySpark
pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
pyspark_custom_imputation_pipeline=pandas_sklearn_pipeline.pyspark_custom_imputation_pipeline(df=pyspark_df, 
                                                                                              output_schema=pyspark_custom_imputation_schema,
                                                                                              analysis_group=analysis_group)
pyspark_custom_imputation_pipeline.show(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+--------------------+-----+-------------------+------------------+--------+-------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|
+--------------------+-----+-------------------+------------------+--------+-------+
|9Fob2bu+MwNd3P522...|190.0|2022-09-13 19:39:18|        2022-09-13|       0|   null|
+--------------------+-----+-------------------+------------------+--------+-------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

### 3. PySpark Aggregate Data at Level

### 4. PySpark: Adding Binary Labels

In [5]:
pyspark_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=pyspark_custom_imputation_pipeline, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

pyspark_df_added_binary_labels.show(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

+--------------------+-----+-------------------+------------------+--------+-------+--------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|y_Binary|
+--------------------+-----+-------------------+------------------+--------+-------+--------+
|9Fob2bu+MwNd3P522...|190.0|2022-09-13 19:39:18|        2022-09-13|       0|   null|       1|
+--------------------+-----+-------------------+------------------+--------+-------+--------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

### 5. PySpark: Features: Summary Statistics

In [6]:
####### PySpark
pyspark_features_summary_stats_schema=pandas_udf_data_schema.summary_stats_schema()
pyspark_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=pyspark_df_added_binary_labels, 
                                                                                 output_schema=pyspark_features_summary_stats_schema, 
                                                                                 lower=daily_stats_features_lower, 
                                                                                 upper=daily_stats_features_upper,
                                                                                 analysis_group=analysis_group)

pyspark_features_summary_stats.show(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+---------+-----+-----+---------+---------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|y_Binary|Median|     Mean|  Std Dev|  Max|  Min|AreaBelow|AreaAbove|
+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+---------+-----+-----+---------+---------+
|9Fob2bu+MwNd3P522...|190.0|2022-09-13 19:39:18|        2022-09-13|       0|   null|       1| 180.0|181.32076|10.229025|202.0|157.0|      0.0|    486.0|
+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+---------+-----+-----+---------+---------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1


### 6. PySpark: Wrapper Functions

### 7. PySpark: Sklearn Categorical Pipeline in PySpark

In [7]:
####### PySpark
pyspark_categorical_schema=pandas_udf_data_schema.sklearn_pyspark_categorical_schema()
pyspark_transformations_categorical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_categorical(df=pyspark_features_summary_stats, 
                                                                                                 output_schema=pyspark_categorical_schema,
                                                                                                 analysis_group=analysis_group)
pyspark_transformations_categorical.show(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data[time_feats] = row_data[time_feats] + np.timedelta64(5*(step+1), 'm')
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

23/04/17 16:40:16 ERROR Executor: Exception in task 0.0 in stage 21.0 (TID 10)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/sklearn_pipeline.py", line 63, in transform_features
    transformed_data_df.columns=['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate',
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/generic.py", line 5916, in __setattr__
    except AttributeError:
  File "pandas/_libs/properties.pyx", line 69, in pandas._libs.properties.AxisProperty.__set__
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/generic.py", line 823, in _set_axis
    self._mgr.set_axis(axis, labels)
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/internals/managers.py", line 230, i

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/sklearn_pipeline.py", line 63, in transform_features
    transformed_data_df.columns=['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate',
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/generic.py", line 5916, in __setattr__
    except AttributeError:
  File "pandas/_libs/properties.pyx", line 69, in pandas._libs.properties.AxisProperty.__set__
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/generic.py", line 823, in _set_axis
    self._mgr.set_axis(axis, labels)
  File "/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pandas/core/internals/managers.py", line 230, in set_axis
    self._v

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [ ]:
####### PySpark
pyspark_numerical_schema=pandas_udf_data_schema.sklearn_pyspark_numerical_schema()
pyspark_transformations_numerical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_numerical(df=pyspark_transformations_categorical, 
                                                                                             output_schema=pyspark_numerical_schema)
pyspark_transformations_numerical.show(1)

### 9. PySpark: Lag Features

In [ ]:
####### PySpark
pyspark_lag_features_creation=create_lag_features.pyspark_lag_features(df=pyspark_transformations_numerical,
                                                                       time_series_lag_values_created=time_series_lag_values_created)
pyspark_lag_features_creation.show(5)

### 10. PySpark: XGBoost Model

In [ ]:
####### PySpark
pyspark_xgboost_classsification_schema=pandas_udf_data_schema.xgboost_classification_schema()

classification_model_outputs=xgboost_classification.pyspark_xgboost(df=pyspark_lag_features_creation, 
                                                                    output_schema=pyspark_xgboost_classsification_schema, 
                                                                    train_split=ml_models_train_split, 
                                                                    test_split=ml_models_test_split)

classification_model_outputs.show()

### 11. PySpark: Model Evaluation

In [ ]:
####### PySpark
pyspark_classification_metric_schema=pandas_udf_data_schema.classification_metric_schema()

classification_metric_df=classification_evalaution_metrics.pyspark_classification_model_evaluation_metrics(df=classification_model_outputs, 
                                                                                                           output_schema=pyspark_classification_metric_schema)
classification_metric_df.show()                    

### 12. PySpark: XGBoost Classification Feature Importance

In [ ]:
xgboost_classification_plot.read_model_plot_variance(model_storage_location=model_storage_location)

# Pandas

### Pandas: Reading in Data

In [8]:
####### Pandas
pandas_df=reading_data.read_in_pandas()
pandas_df.head(1)

,PostDate,IngestionDate,PostId,PostTime,PatientId,Stream,SequenceNumber,TransmitterNumber,ReceiverNumber,RecordedSystemTime,...,GlucoseSystemTime,GlucoseDisplayTime,GlucoseDisplayTimeRaw,Value,Status,TrendArrow,TrendRate,IsBackFilled,InternalStatus,SessionStartTime
0,2022-09-14,2022-09-14,fyQ0wOxwB8sthzC75TWKpqdrCMCLq+W1wBg9s7MHVcE=,2022-09-14 00:26:00,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,Phone7,1663115129,XFgG633aV9tw5Gclf1WB6nmt9Wgv4nHJjXfRQIAOfsg=,NaN,2022-09-13 23:16:01,...,2022-09-13 23:15:45,2022-09-13 19:15:45,2022-09-13 19:15:45-04:00,111.0,NaN,Flat,0.2,FALSE,6,5137335


### Pandas: Custom Imputation Pipeline

In [ ]:
test_pat=pandas_df[pandas_df['PatientId']=='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=']

In [9]:
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1


,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,NaN


In [10]:
####### Pandas
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1


,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,NaN


### Pandas: Aggregate Data at Level

### Pandas: Adding Binary Labels

In [11]:
pandas_df_added_binary_labels=create_binary_labels.pandas_binary_labels(df=pandas_custom_imputation_pipeline, 
                                                                        lower=daily_stats_features_lower, 
                                                                        upper=daily_stats_features_upper)
pandas_df_added_binary_labels.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing,y_Binary
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,NaN,0


### Pandas: Features: Summary Statistics

In [12]:
pandas_features_summary_stats=summary_stats_features.pandas_compressDailyValues(data=pandas_df_added_binary_labels, 
                                                                                lower=daily_stats_features_lower, 
                                                                                upper=daily_stats_features_upper)
pandas_features_summary_stats.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,y_Binary,inserted,missing,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,0,NaN,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0


### Pandas: Wrapper Functions

### Pandas: Sklearn Categorical Pipeline in Pandas

In [15]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
df=pandas_features_summary_stats[['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 'inserted', 
        'missing', 'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 'AreaAbove']]

In [16]:
df.head()

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing,y_Binary,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,NaN,0,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0
1,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,109.0,2022-09-13 19:20:45,2022-09-13,0,NaN,0,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0
2,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:25:45,2022-09-13,0,NaN,0,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0
3,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:30:45,2022-09-13,0,NaN,0,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0
4,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,110.0,2022-09-13 19:35:45,2022-09-13,0,NaN,0,204.0,198.175439,44.671559,256.0,109.0,0.0,3394.0


In [23]:
for patient_id in df['PatientId'].unique():
    # Categorical Features
    categorical_features=['inserted', 'missing']
    categorical_transformer=Pipeline([('imputer_cat', SimpleImputer(strategy='constant', fill_value=np.nan)),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor_2=ColumnTransformer([('categorical', categorical_transformer, categorical_features)],
                                    remainder = 'passthrough')

    cat_pipe_pipeline=Pipeline([('preprocessing_2', preprocessor_2)])

    transformed_data1=cat_pipe_pipeline.fit_transform(df)

    transformed_data_df=pd.DataFrame(transformed_data1)

    transformed_data_df['combine_inserted']=transformed_data_df[[0,1]].values.tolist()
    transformed_data_df['combine_missing']=transformed_data_df[[2,3]].values.tolist()
    transformed_data_df=transformed_data_df.drop(transformed_data_df.iloc[:, 0:4],axis = 1)

    transformed_data_df.columns=['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 
                                    'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 
                                    'AreaAbove', 'inserted', 'missing']

                                         PatientId       Value  \
0     tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=       111.0   
1     tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=       109.0   
2     tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=       111.0   
3     tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=       111.0   
4     tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=       110.0   
...                                            ...         ...   
3435  iSFLmNjVJ3w4e4WcQtF3eMEKXOa4IKaeDjZ7Lra7l+Y=  185.083152   
3436  9Fob2bu+MwNd3P522PazJ5pIV4l3jvx9Tf98DfxSdoU=       254.0   
3437  9Fob2bu+MwNd3P522PazJ5pIV4l3jvx9Tf98DfxSdoU=       251.0   
3438  9Fob2bu+MwNd3P522PazJ5pIV4l3jvx9Tf98DfxSdoU=       245.0   
3439  9Fob2bu+MwNd3P522PazJ5pIV4l3jvx9Tf98DfxSdoU=       245.0   

      GlucoseDisplayTime GlucoseDisplayDate y_Binary Median        Mean  \
0    2022-09-13 19:15:45         2022-09-13        0  204.0  198.175439   
1    2022-09-13 19:20:45         2022-09-13        0  204

In [25]:
transformed_data_df.columns

Index(['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate',
       'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow',
       'AreaAbove', 'inserted', 'missing'],
      dtype='object')

In [ ]:
####### Pandas
pandas_custom_categorical_pipeline=pandas_sklearn_pipeline.pandas_transform_categorical_features(df=pandas_features_summary_stats)
pandas_custom_categorical_pipeline.head(1)

### Pandas: Sklearn Numerical Pipeline in Pandas

In [ ]:
####### Pandas
pandas_custom_numerical_pipeline=pandas_sklearn_pipeline.pandas_transform_numerical_features(df=pandas_custom_categorical_pipeline)
pandas_custom_numerical_pipeline.head(1)

### Pandas: Models